<a href="https://colab.research.google.com/github/AdvancedUno/HuBMAP-HPA---Hacking-the-Human-Body-Kaggle-/blob/main/kaggle_data_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
import os 

In [ ]:
path = '/content/drive/MyDrive/DeepLearning/train_images' # 폴더 경로
os.chdir(path) # 해당 폴더로 이동
files = os.listdir(path) # 해당 폴더에 있는 파일 이름을 

In [ ]:
i  = 0
for file in files:
      newfile = file.replace("tiff", "png")
      print(newfile)
      
      i+=1
      f = cv2.imread(file)
      img = cv2.resize(f, dsize=(640, 640), interpolation=cv2.INTER_CUBIC)
      cv2.imwrite("/content/drive/MyDrive/DeepLearning/training/"+newfile, img)

print(i)

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
ls -lha kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c hubmap-organ-segmentation

In [ ]:
!unzip -qq "/content/drive/MyDrive/DeepLearning/hubmap-organ-segmentation.zip"

In [ ]:
import pandas as pd
import numpy as np

import os
import json
from tqdm.auto import tqdm
import gc

from skimage import io
from skimage.transform import resize

In [ ]:
def rle2mask(mask_rle: str, shape=None, label: int = 0):
    """
    mask_rle: run-length as string formatted (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background

    """
    rle = np.array(list(map(int, mask_rle.split())))
    labels = np.zeros(shape).flatten()
    
    for start, end in zip(rle[::2], rle[1::2]):
        labels[start:start+end] = label

    return labels.reshape(shape).T  # Needed to align to RLE direction

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/DeepLearning/train.csv")

In [ ]:
organs = df_train['organ'].unique()

In [ ]:
organ_annotations = {}

for i, organ in enumerate(organs):
    organ_annotations[organ] = i+1
    
organ_annotations

In [ ]:
from google.colab.patches import cv2_imshow
for file in files:
      newfile = file.replace(".tiff", "")
      savefile = file.replace("tiff", "png")
      #print(newfile)
      #print(df_train)
      newfile = int(newfile)
      
      rle = df_train[df_train['id'] == newfile]['rle'].values[0]
      height = df_train[df_train['id'] == newfile]['img_height'].values[0]
      width = df_train[df_train['id'] == newfile]['img_width'].values[0]
      organ = df_train[df_train['id'] == newfile]['organ'].values[0]
      
      mask = rle2mask(rle, shape=(height, width), label=organ_annotations[organ])
      mask = cv2.resize(mask, dsize=(640, 640), interpolation=cv2.INTER_CUBIC)
      cv2.imwrite("/content/drive/MyDrive/DeepLearning/mask/"+savefile, mask)
      
      #cv2.imshow("a", mask)
      #print(mask.shape)
      #cv2_imshow(mask)

      #cv2.waitKey()

      #cv2.destroyAllWindows()

